In [1]:
import os
import time
import certifi
import requests
import pandas as pd
import xml.etree.ElementTree as ET


In [2]:
base_url = catalog.load('params:oai_extract_options.base_url')
context = catalog.load('params:oai_extract_options.context')

env = 'dev'

print("base_url: ", base_url)
print("context: ", context)

                    INFO     Loading data from params:oai_extract_options.base_url              ]8;id=50168;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=921516;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_extract_options.context               ]8;id=832186;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=913474;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

base_url:  https://ri.conicet.gov.ar/oai/
context:  request


In [3]:
def get_oai_response(base_url, verify=None, max_retries=3, backoff_factor=1.0):

    # Usa el bundle de certifi para evitar errores de certificado en requests
    os.environ.setdefault("REQUESTS_CA_BUNDLE", certifi.where())
    os.environ.setdefault("SSL_CERT_FILE", certifi.where())
    VERIFY_SSL = os.getenv("OAI_VERIFY_SSL", "false").lower() == "true"
    CA_BUNDLE = os.getenv("OAI_CA_BUNDLE") or certifi.where()

    verify_param = CA_BUNDLE if VERIFY_SSL else False
    if verify is not None:
        verify_param = verify

    for attempt in range(1, max_retries + 1):
        start_time = time.time()
        response = None
        try:
            response = requests.get(base_url, verify=verify_param)
            elapsed_time = time.time() - start_time
        except requests.RequestException as exc:
            elapsed_time = time.time() - start_time
            print(f"Error en request (intento {attempt}/{max_retries}): {exc}")
        sleep_time = max(elapsed_time, 0.1)
        print(f"Sleeping for {sleep_time:.2f} seconds")
        time.sleep(sleep_time)

        if response and response.status_code == 200:
            return response

        status = response.status_code if response else "sin respuesta"
        print(f"Error: {status} (intento {attempt}/{max_retries})")

        if attempt < max_retries:
            backoff = backoff_factor * attempt
            print(f"Reintentando en {backoff:.2f} segundos...")
            time.sleep(backoff)
    return None


In [4]:
params = f'/{context}?verb=ListSets&resumptionToken=////0'
url = base_url + params

print(f"Consultando: {url}")

response = get_oai_response(url)

xml_content = response.text
root = ET.fromstring(xml_content)
ns = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

root

Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListSets&resumptionToken=////0


[11/27/25 12:14:07] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=425385;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=679927;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.33 seconds


<Element '{http://www.openarchives.org/OAI/2.0/}OAI-PMH' at 0x700450ac7d30>

In [5]:
root.find('.//oai:resumptionToken', ns).text

'////100'

In [6]:
token_elem = root.find('.//oai:resumptionToken', ns)
if token_elem is not None:
    complete_list_size = token_elem.get('completeListSize')
    cursor = token_elem.get('cursor')
    resumptionToken = token_elem.text

In [7]:
complete_list_size

'2444'

In [8]:
sets_data = []
for set_elem in root.findall('.//oai:set', ns):
    set_spec = set_elem.find('oai:setSpec', ns).text if set_elem.find('oai:setSpec', ns) is not None else None
    set_name = set_elem.find('oai:setName', ns).text if set_elem.find('oai:setName', ns) is not None else None
    sets_data.append({'setSpec': set_spec, 'setName': set_name})
sets_data


[
    {'setSpec': 'snrd', 'setName': 'Sistema Nacional de Repositorios Digitales'},
    {
        'setSpec': 'com_11336_73',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - BAHIA BLANCA'
    },
    {
        'setSpec': 'com_11336_116',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - CORDOBA'
    },
    {
        'setSpec': 'com_11336_169',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - LA PLATA'
    },
    {
        'setSpec': 'com_11336_184',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - MAR DEL PLATA'
    },
    {
        'setSpec': 'com_11336_199',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - MENDOZA'
    },
    {
        'setSpec': 'com_11336_330',
        'setName': 'AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGICO CONICET - NOA SUR'
    },
    {
        'setSpec': 'com_11336_216',
        'setName': 'AREA DE INFLUENCIA C